# 2021/11/1
# 本代码计算整层积分MSE
# 使用scipy，一来科研，也是学习

In [1]:
from scipy import integrate
import numpy as np
import xarray as xr

In [2]:
path  =  "/home/sun/data/"
f     =  xr.open_dataset(path+"merra2_climate_pentad_mse.nc").sel(lev=slice(1000,100))
lev   =  f.lev.data

In [4]:
integrate_mse   =   np.zeros((f.mse.data.shape[0],f.mse.data.shape[2],f.mse.data.shape[3]))
for pen in range(0,f.mse.data.shape[0]):
    for lat in range(0,f.mse.data.shape[2]):
        for lon in range(0,f.mse.data.shape[3]):
            trans  =  np.nan_to_num(f.mse.data[pen,:,lat,lon],nan=0)
            integrate_mse[pen,lat,lon]  =  -1*integrate.trapz(trans,lev)/9.8


In [5]:
ncfile  =  xr.Dataset(
    {
        "integrated_mse": (["time", "lat", "lon"], integrate_mse),
    },
    coords={
        "lon": (["lon"], f.lon.data),
        "lat": (["lat"], f.lat.data),
        "time": (["time"], np.linspace(1,73,73)),
    },
)
ncfile["lat"].attrs  =  f["lat"].attrs
ncfile["lon"].attrs  =  f["lon"].attrs
ncfile["integrated_mse"].attrs["units"]  =  "kj/kg"

ncfile.to_netcdf("/home/sun/data/merra2_integrated_mse.nc")